In [20]:
from nltk.stem import WordNetLemmatizer
# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from nltk.tokenize import TweetTokenizer
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import numpy as np

In [2]:
from sklearn import model_selection, preprocessing, metrics, linear_model, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from imblearn.over_sampling import BorderlineSMOTE, SMOTE, ADASYN, SMOTENC, RandomOverSampler
from imblearn.under_sampling import (RandomUnderSampler, 
                                    NearMiss, 
                                    InstanceHardnessThreshold,
                                    CondensedNearestNeighbour,
                                    EditedNearestNeighbours,
                                    RepeatedEditedNearestNeighbours,
                                    AllKNN,
                                    NeighbourhoodCleaningRule,
                                    OneSidedSelection,
                                    TomekLinks)
from imblearn.combine import SMOTEENN, SMOTETomek

In [3]:
data =pd.read_csv("Tweets.csv")

In [4]:
data.head()

,tweet_id,unclean_tweets,clean_tweets,first100charactersoftweets,polarity,subjectivity,sentiment,neg,neu,pos,compound,clean_tweets_length,word_count,non_punctuated_tweets
0,1477985792605757447,@OctoSquid_503 I put about 50-60 hours into th...,put hours game last year still holds well blas...,put hours game last year still holds well blas...,-0.047619,0.545238,positive,0.000,0.815,0.185,0.6124,122,20,put hours game last year still holds well blas...
1,1477984041248989187,A story about why 2021 was great and not terri...,story great terrible,story great terrible,-0.100000,0.875000,positive,0.165,0.588,0.248,0.1803,20,3,story great terrible
2,1477983892665688069,The jaedo event I bought tickets for in the 2n...,jaedo event bought tickets nd quarter finally ...,jaedo event bought tickets nd quarter finally ...,-0.333333,1.000000,negative,0.230,0.556,0.214,-0.0516,114,18,jaedo event bought tickets nd quarter finally ...
3,1477981564323569664,this year is going to be super dynamic for me ...,year going super dynamic bc im graduating uni ...,year going super dynamic bc im graduating uni ...,0.130556,0.477778,positive,0.000,0.772,0.228,0.6808,178,31,year going super dynamic bc im graduating uni ...
4,1477981357535944707,2021 literally such a tough year 😭 none of the...,literally tough year none plan going right pre...,literally tough year none plan going right pre...,0.005423,0.684127,positive,0.000,0.843,0.157,0.3818,112,20,literally tough year none plan going right pre...


In [5]:
Data=data[['non_punctuated_tweets','sentiment']]

In [6]:
Data.non_punctuated_tweets[0]

'put hours game last year still holds well blast playing definitely one highlights great still active sad day servers empty'

In [7]:
Data

,non_punctuated_tweets,sentiment
0,put hours game last year still holds well blas...,positive
1,story great terrible,positive
2,jaedo event bought tickets nd quarter finally ...,negative
3,year going super dynamic bc im graduating uni ...,positive
4,literally tough year none plan going right pre...,positive
...,...,...
17769,really rough one mental health truly feel many...,positive
17770,happy new year everyone except boy dated coupl...,positive
17771,crazy bought car moved shot porn butt plug bum...,negative
17772,think descent art year despite able use tablet...,neutral


In [8]:
Data['sentiment'] = Data['sentiment'].replace('positive',1)
Data['sentiment'] = Data['sentiment'].replace('negative',-1)
Data['sentiment'] = Data['sentiment'].replace('neutral',0)

### code for encoding the meaning of positive negative and  neutral tweets

C:\Users\Olazaah\Anaconda3\envs\tensorlow1\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Olazaah\Anaconda3\envs\tensorlow1\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Olazaah\Anaconda3\envs\tensorlow1\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_in

In [9]:
Data.sentiment

0        1
1        1
2       -1
3        1
4        1
        ..
17769    1
17770    1
17771   -1
17772    0
17773    1
Name: sentiment, Length: 17774, dtype: int64

In [10]:
Data['sentiment'].unique()

array([ 1, -1,  0], dtype=int64)

In [11]:
Data['sentiment'].value_counts()

 1    12622
-1     3823
 0     1329
Name: sentiment, dtype: int64

In [12]:
 Data['non_punctuated_tweets']

0        put hours game last year still holds well blas...
1                                     story great terrible
2        jaedo event bought tickets nd quarter finally ...
3        year going super dynamic bc im graduating uni ...
4        literally tough year none plan going right pre...
                               ...                        
17769    really rough one mental health truly feel many...
17770    happy new year everyone except boy dated coupl...
17771    crazy bought car moved shot porn butt plug bum...
17772    think descent art year despite able use tablet...
17773    hi active moment wanted say happy new year acc...
Name: non_punctuated_tweets, Length: 17774, dtype: object

In [21]:
# Create a reference variable for Class TweetTokenizer
tt = TweetTokenizer()

In [25]:
#from nltk.tokenize import RegexpTokenizer
#tokenizer = RegexpTokenizer(r'w+')
Data['tokenized_tweets'] = Data['non_punctuated_tweets'].apply(tt.tokenize)
#Data['tokenized_tweets'].head()

TypeError: expected string or buffer

In [17]:
Data

,non_punctuated_tweets,sentiment
0,put hours game last year still holds well blas...,1
1,story great terrible,1
2,jaedo event bought tickets nd quarter finally ...,-1
3,year going super dynamic bc im graduating uni ...,1
4,literally tough year none plan going right pre...,1
...,...,...
17769,really rough one mental health truly feel many...,1
17770,happy new year everyone except boy dated coupl...,1
17771,crazy bought car moved shot porn butt plug bum...,-1
17772,think descent art year despite able use tablet...,0


In [61]:
import nltk
st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return text
Data['stemmed_tweets']= Data['tokenized_tweets'].apply(lambda x: stemming_on_text(x))
Data['stemmed_tweets'].head()

C:\Users\Olazaah\Anaconda3\envs\tensorlow1\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0    [put, hour, game, last, year, still, hold, wel...
1                              [stori, great, terribl]
2    [jaedo, event, bought, ticket, nd, quarter, fi...
3    [year, go, super, dynam, bc, im, graduat, uni,...
4    [liter, tough, year, none, plan, go, right, pr...
Name: stemmed_tweets, dtype: object

In [62]:
X=Data.non_punctuated_tweets
y=Data.sentiment

In [66]:
y = np.array(y)

In [100]:
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state =26105111)

# converting tweets to vectors using TFID

In [71]:
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X)
X_train_tfidf =  tfidf_vect.transform(X_train)
X_test_tfidf =  tfidf_vect.transform(X_test)

# Modelling the data

In [182]:
#Return the f1 Score
def train_model(name_of_classifier,classifier, feature_vector_train, label, feature_vector_test):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_test)
    f1_micro = metrics.f1_score(y_test,predictions,average = 'micro')
    f1_macro = metrics.f1_score(y_test,predictions,average = 'macro')
    f1_weighted = metrics.f1_score(y_test,predictions,average = 'weighted')
    accuracy = accuracy_score(y_test,predictions)
    return pd.DataFrame({'model':[name_of_classifier],'f1_micro':[f1_micro],'f1_macro':[f1_macro],'f1_weighted':[f1_weighted],'accuracy':[accuracy]})

# without resampling the imbalanced Dataset

In [183]:
first = train_model('logistc_regression',LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),X_train_tfidf, y_train, X_test_tfidf)
#print ("LR Baseline, WordLevel TFIDF: ", accuracyORIGINAL)
second = train_model('SVM',LinearSVC(),X_train_tfidf, y_train, X_test_tfidf)
third =  train_model('BernoulliNB',BernoulliNB(),X_train_tfidf, y_train, X_test_tfidf)
fourth =  train_model('MultinomialNB',MultinomialNB(),X_train_tfidf, y_train, X_test_tfidf)

frames = [first,second,third,fourth]
combined = pd.concat(frames)
combined.reset_index(drop=True, inplace=True)
display(combined)

C:\Users\Olazaah\Anaconda3\envs\tensorlow1\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


,model,f1_micro,f1_macro,f1_weighted,accuracy
0,logistc_regression,0.715049,0.367108,0.640890,0.715049
1,SVM,0.691139,0.404903,0.647170,0.691139
2,BernoulliNB,0.688608,0.351600,0.627255,0.688608
3,MultinomialNB,0.712799,0.280867,0.596135,0.712799


In [ ]:
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.model_selection import train_test_split
#model=BernoulliNB()
#model.fit(X_train_tfidf, y_train)


# Random OverSampling

In [112]:
#Random Over Sampling
ros = RandomOverSampler(random_state=777)
ros_X_train_tfidf, ros_y_train = ros.fit_resample(X_train_tfidf, y_train)

In [113]:
first = train_model('logistc_regression',LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),ros_X_train_tfidf, ros_y_train, X_test_tfidf)
#print ("LR Baseline, WordLevel TFIDF: ", accuracyORIGINAL)
second = train_model('SVM',LinearSVC(),ros_X_train_tfidf, ros_y_train, X_test_tfidf)
third =  train_model('BernoulliNB',BernoulliNB(),ros_X_train_tfidf, ros_y_train, X_test_tfidf)
fourth =  train_model('MultinomialNB',MultinomialNB(),ros_X_train_tfidf, ros_y_train, X_test_tfidf)

frames = [first,second,third,fourth]
combined = pd.concat(frames)
combined.reset_index(drop=True, inplace=True)
display(combined)

C:\Users\Olazaah\Anaconda3\envs\tensorlow1\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


,model,f1_micro,f1_macro,f1_weighted,accuracy
0,logistc_regression,0.602813,0.440973,0.620707,0.602813
1,SVM,0.604782,0.417512,0.617462,0.604782
2,BernoulliNB,0.562025,0.425878,0.595448,0.562025
3,MultinomialNB,0.600844,0.449748,0.621284,0.600844


# SMOTE Over-sampling

In [115]:
sm = SMOTE(random_state=777)
sm_X_train_tfidf, sm_y_train = sm.fit_resample(X_train_tfidf, y_train)

In [116]:
first = train_model('logistc_regression',LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),sm_X_train_tfidf, sm_y_train, X_test_tfidf)
#print ("LR Baseline, WordLevel TFIDF: ", accuracyORIGINAL)
second = train_model('SVM',LinearSVC(),sm_X_train_tfidf, sm_y_train, X_test_tfidf)
third =  train_model('BernoulliNB',BernoulliNB(),sm_X_train_tfidf, sm_y_train, X_test_tfidf)
fourth =  train_model('MultinomialNB',MultinomialNB(),sm_X_train_tfidf, sm_y_train, X_test_tfidf)

frames = [first,second,third,fourth]
combined = pd.concat(frames)
combined.reset_index(drop=True, inplace=True)
display(combined)

C:\Users\Olazaah\Anaconda3\envs\tensorlow1\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


,model,f1_micro,f1_macro,f1_weighted,accuracy
0,logistc_regression,0.602250,0.435314,0.618899,0.602250
1,SVM,0.593812,0.412383,0.608538,0.593812
2,BernoulliNB,0.645288,0.410574,0.631968,0.645288
3,MultinomialNB,0.579747,0.437375,0.605700,0.579747


# ADASYN: Adaptive synthetic sampling (Over-sampling)

In [118]:
#ADASYN
ad = ADASYN(random_state=777)
ad_xtrain_tfidf, ad_train_y = ad.fit_resample(X_train_tfidf, y_train)

In [119]:
first = train_model('logistc_regression',LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),ad_xtrain_tfidf, ad_train_y, X_test_tfidf)
#print ("LR Baseline, WordLevel TFIDF: ", accuracyORIGINAL)
second = train_model('SVM',LinearSVC(),ad_xtrain_tfidf, ad_train_y, X_test_tfidf)
third =  train_model('BernoulliNB',BernoulliNB(),ad_xtrain_tfidf, ad_train_y, X_test_tfidf)
fourth =  train_model('MultinomialNB',MultinomialNB(),ad_xtrain_tfidf, ad_train_y, X_test_tfidf)

frames = [first,second,third,fourth]
combined = pd.concat(frames)
combined.reset_index(drop=True, inplace=True)
display(combined)

C:\Users\Olazaah\Anaconda3\envs\tensorlow1\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


,model,f1_micro,f1_macro,f1_weighted,accuracy
0,logistc_regression,0.600844,0.438161,0.618236,0.600844
1,SVM,0.588748,0.410657,0.605510,0.588748
2,BernoulliNB,0.643601,0.406235,0.629260,0.643601
3,MultinomialNB,0.587342,0.436590,0.611706,0.587342


# BorderLine SMOTE

In [120]:
bsm = BorderlineSMOTE()
bsm_xtrain_tfidf, bsm_train_y = bsm.fit_resample(X_train_tfidf, y_train)

In [121]:
first = train_model('logistc_regression',LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),bsm_xtrain_tfidf, bsm_train_y, X_test_tfidf)
#print ("LR Baseline, WordLevel TFIDF: ", accuracyORIGINAL)
second = train_model('SVM',LinearSVC(),bsm_xtrain_tfidf, bsm_train_y, X_test_tfidf)
third =  train_model('BernoulliNB',BernoulliNB(),bsm_xtrain_tfidf, bsm_train_y, X_test_tfidf)
fourth =  train_model('MultinomialNB',MultinomialNB(),bsm_xtrain_tfidf, bsm_train_y, X_test_tfidf)

frames = [first,second,third,fourth]
combined = pd.concat(frames)
combined.reset_index(drop=True, inplace=True)
display(combined)

C:\Users\Olazaah\Anaconda3\envs\tensorlow1\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


,model,f1_micro,f1_macro,f1_weighted,accuracy
0,logistc_regression,0.612377,0.439430,0.625027,0.612377
1,SVM,0.600563,0.414806,0.612941,0.600563
2,BernoulliNB,0.649789,0.415879,0.634865,0.649789
3,MultinomialNB,0.595781,0.450686,0.618201,0.595781


# Random Under-sampling

In [122]:
rus = RandomUnderSampler(random_state=0, replacement=True)
rus_xtrain_tfidf, rus_train_y = rus.fit_resample(X_train_tfidf, y_train)

In [124]:
first = train_model('logistc_regression',LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),rus_xtrain_tfidf, rus_train_y ,X_test_tfidf)
#print ("LR Baseline, WordLevel TFIDF: ", accuracyORIGINAL)
second = train_model('SVM',LinearSVC(),rus_xtrain_tfidf, rus_train_y, X_test_tfidf)
third =  train_model('BernoulliNB',BernoulliNB(),rus_xtrain_tfidf, rus_train_y, X_test_tfidf)
fourth =  train_model('MultinomialNB',MultinomialNB(),rus_xtrain_tfidf, rus_train_y, X_test_tfidf)

frames = [first,second,third,fourth]
combined = pd.concat(frames)
combined.reset_index(drop=True, inplace=True)
display(combined)

,model,f1_micro,f1_macro,f1_weighted,accuracy
0,logistc_regression,0.450352,0.377926,0.505146,0.450352
1,SVM,0.432911,0.360309,0.488528,0.432911
2,BernoulliNB,0.409283,0.340443,0.472285,0.409283
3,MultinomialNB,0.556399,0.422520,0.586896,0.556399


# Tomek Link Removal

In [125]:
tl = TomekLinks()
tl_xtrain_tfidf, tl_train_y = tl.fit_resample(X_train_tfidf, y_train)

In [126]:
first = train_model('logistc_regression',LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),tl_xtrain_tfidf, tl_train_y ,X_test_tfidf)
#print ("LR Baseline, WordLevel TFIDF: ", accuracyORIGINAL)
second = train_model('SVM',LinearSVC(),tl_xtrain_tfidf, tl_train_y , X_test_tfidf)
third =  train_model('BernoulliNB',BernoulliNB(),tl_xtrain_tfidf, tl_train_y, X_test_tfidf)
fourth =  train_model('MultinomialNB',MultinomialNB(),tl_xtrain_tfidf, tl_train_y , X_test_tfidf)

frames = [first,second,third,fourth]
combined = pd.concat(frames)
combined.reset_index(drop=True, inplace=True)
display(combined)

C:\Users\Olazaah\Anaconda3\envs\tensorlow1\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


,model,f1_micro,f1_macro,f1_weighted,accuracy
0,logistc_regression,0.719550,0.371198,0.643053,0.719550
1,SVM,0.693671,0.403787,0.648962,0.693671
2,BernoulliNB,0.691702,0.346016,0.625138,0.691702
3,MultinomialNB,0.713361,0.279274,0.595293,0.713361
